<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
import sys
sys.path.insert(0, os.path.relpath('../../../../..')) 



In [2]:
import pandas as pd
import numpy as np
from recidiviz.calculator.modeling.population_projection.utils.spark_bq_utils import (
    upload_spark_model_inputs,
)
from recidiviz.calculator.modeling.population_projection.utils.spark_preprocessing_utils import (
    yearly_to_monthly_data
)

In [3]:
raw_va_sentence_df = pd.read_csv(
    '../state/VA/VA_data/unprocessed_va_historical_sentences_v2.csv',
    sep='\t'
)
raw_va_sentence_df['crime_type'] = raw_va_sentence_df['Offense Group'].ffill()
raw_va_sentence_df['offense_code'] = raw_va_sentence_df['VCC'].ffill()
raw_va_sentence_df['crime'] = raw_va_sentence_df['Off1VCC'].ffill()
raw_va_sentence_df['judge_id'] = raw_va_sentence_df['JudgeID'].ffill()
raw_va_sentence_df['sentence_type_code'] = raw_va_sentence_df['ActDisp'].ffill()
raw_va_sentence_df['effective_sentence_months'] = raw_va_sentence_df['effsent']
raw_va_sentence_df['fiscal_year'] = raw_va_sentence_df['FiscalYr'].ffill()
raw_va_sentence_df['life_sentence'] = raw_va_sentence_df['EffLif']
raw_va_sentence_df['offense_date'] = raw_va_sentence_df['Off1Date']

raw_va_sentence_df = raw_va_sentence_df[~raw_va_sentence_df['crime_type'].str.contains('Total')]

In [24]:
raw_va_sentence_df

,Offense Group,VCC,Off1VCC,JudgeID,ActDisp,FiscalYr,Off1Date,EffLif,effsent,OffLName,OffFName,crime_type,offense_code,crime,judge_id,sentence_type_code,effective_sentence_months,fiscal_year,life_sentence,offense_date
0,ASSAULT,ASL1316,ASL1316F6,0,3.0,2018.0,5/20/2017,0.0,18.0,MOORE,KENNETH,ASSAULT,ASL1316,ASL1316F6,0,3.0,18.0,2018.0,0.0,5/20/2017
1,NaN,NaN,NaN,NaN,NaN,2019.0,4/25/2018,0.0,10.0,CLINE,MATTHEW,ASSAULT,ASL1316,ASL1316F6,0,3.0,10.0,2019.0,0.0,4/25/2018
2,NaN,NaN,NaN,10DB,1.0,2018.0,11/11/2017,0.0,0.0,TINSLEY,JAVON,ASSAULT,ASL1316,ASL1316F6,10DB,1.0,0.0,2018.0,0.0,11/11/2017
3,NaN,NaN,NaN,NaN,2.0,2017.0,6/8/2015,0.0,0.0,COMPTON,ROBERT,ASSAULT,ASL1316,ASL1316F6,10DB,2.0,0.0,2017.0,0.0,6/8/2015
4,NaN,NaN,NaN,NaN,3.0,2017.0,12/25/2016,0.0,7.0,MILLER,MARCUS,ASSAULT,ASL1316,ASL1316F6,10DB,3.0,7.0,2017.0,0.0,12/25/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74313,NaN,NaN,NaN,9MM,3.0,2018.0,6/1/2017,0.0,8.0,CUNNINGHAM,JACOB,WEAPONS/FIREARMS,WPN5298,WPN5298F6,9MM,3.0,8.0,2018.0,0.0,6/1/2017
74314,NaN,NaN,NaN,NaN,NaN,NaN,6/27/2017,0.0,13.0,STURDIVANT,JUSTIN,WEAPONS/FIREARMS,WPN5298,WPN5298F6,9MM,3.0,13.0,2018.0,0.0,6/27/2017
74315,NaN,NaN,NaN,NaN,NaN,2019.0,11/30/2017,0.0,7.0,MAINES,BRADY,WEAPONS/FIREARMS,WPN5298,WPN5298F6,9MM,3.0,7.0,2019.0,0.0,11/30/2017
74316,NaN,NaN,NaN,9TH,2.0,2018.0,12/26/2016,0.0,5.0,MORRIS,DAVID,WEAPONS/FIREARMS,WPN5298,WPN5298F6,9TH,2.0,5.0,2018.0,0.0,12/26/2016


In [9]:
# disaggregation_axes = ['crime', 'judge_id', 'crime_type']
disaggregation_axes = ['crime_type']

REFERENCE_YEAR = 2020

In [29]:
(raw_va_sentence_df.effective_sentence_months > 0).apply(lambda x: int(x))

0        1
1        1
2        0
3        0
4        1
        ..
74313    1
74314    1
74315    1
74316    1
74317    1
Name: effective_sentence_months, Length: 74302, dtype: int64

In [10]:
# temporary filter
raw_va_sentence_df = raw_va_sentence_df[raw_va_sentence_df.effective_sentence_months > 0]
# set ts to years for now
raw_va_sentence_df['time_step'] = raw_va_sentence_df.fiscal_year - REFERENCE_YEAR

# convert the ActDisp numerical value into an incarceration type flag
act_disp_dict = {1: 'probation', 2: 'jail', 3: 'prison'}
raw_va_sentence_df['sentence_type'] = raw_va_sentence_df['sentence_type_code'].apply(lambda x: act_disp_dict[x])

In [11]:
# OUTFLOWS DATA

outflows_data = raw_va_sentence_df.rename({'sentence_type': 'outflow_to'}, axis=1)

outflows_data = outflows_data.groupby(
    ['time_step', 'outflow_to', 'crime_type'],
    as_index=False
).agg({'OffLName': 'count'})
outflows_data['compartment'] = 'pretrial'
outflows_data = outflows_data.rename({'OffLName': 'total_population'}, axis=1)
# convert to monthly
outflows_data = yearly_to_monthly_data(outflows_data)

In [33]:
pd.set_option('max_columns', None)
outflows_data.head()

,time_step,outflow_to,crime_type,total_population,compartment
0,-36,jail,ASSAULT,22.416667,pretrial
1,-36,jail,BURGLARY/DWELLING,8.250000,pretrial
2,-36,jail,BURGLARY/OTHER,4.000000,pretrial
3,-36,jail,DRUG OTHER,31.833333,pretrial
4,-36,jail,DRUG/SCHEDULE I/II,180.666667,pretrial


In [13]:
# TRANSITIONS DATA
transitions_data = raw_va_sentence_df.rename(
    {'sentence_type': 'compartment', 'effective_sentence_months': 'compartment_duration'},
    axis=1
)

# Don't want sentences listed as hundreds of years to skew our model, so we cap sentence length at 50 years
transitions_data.loc[
    transitions_data.compartment_duration > 50 * 12, "compartment_duration"
] = 50 * 12

transitions_data = transitions_data.groupby(
    ['compartment', 'compartment_duration'] + disaggregation_axes,
    as_index=False
).agg({'OffLName': 'count'})

transitions_data['outflow_to'] = 'release'
transitions_data = transitions_data.rename({'OffLName': 'total_population'}, axis=1)
transitions_data.total_population = transitions_data.total_population.astype(float)

In [14]:
# for each sub-simulation, add in trivial transitions data to define release behavior
if disaggregation_axes:
    for subgroup in transitions_data.groupby(disaggregation_axes).count().index:
        if len(disaggregation_axes) == 1:
            disaggregation_data = {disaggregation_axes[0]: subgroup}
        else:
            disaggregation_data = {disaggregation_axes[i]: subgroup[i] for i in range(len(disaggregation_axes))}
        transitions_data = transitions_data.append(
            {
                **disaggregation_data,
                "compartment": "release",
                "compartment_duration": 360,
                "total_population": 1,
                "outflow_to": "release",
            },
            ignore_index=True,
        )
else:
    # if no disaggregation, need to add a placeholder disaggregation axis
    outflows_data['crime_type'] = 'NA'
    transitions_data['crime_type'] = 'NA'
    transitions_data = transitions_data.append(
        {
            "crime_type": "NA",
            "compartment": "release",
            "compartment_duration": 360,
            "total_population": 1,
            "outflow_to": "release",
        },
        ignore_index=True,
    )

In [15]:
# filter out infrequent offenses
counts = outflows_data.groupby('crime_type').sum().total_population
counts = counts[counts >= 50]
outflows_data = outflows_data[outflows_data.crime_type.apply(lambda x: x in counts.index)]

if disaggregation_axes:
    if len(disaggregation_axes) == 1:
        outflows_subgroups = list(outflows_data.groupby(disaggregation_axes).count().index)
        transitions_data = transitions_data[
            transitions_data.apply(lambda x: x[disaggregation_axes][0] in outflows_subgroups, axis=1)
        ]
    else:
        outflows_subgroups = [list(i) for i in outflows_data.groupby(disaggregation_axes).count().index]
        transitions_data = transitions_data[
            transitions_data.apply(lambda x: list(x[disaggregation_axes]) in outflows_subgroups, axis=1)
        ]


In [16]:
# Ensure every subgroup has at least one datapoint for jail and prison
if disaggregation_axes:
    subgroups = transitions_data.groupby(disaggregation_axes).count().index
    for subgroup in subgroups:
        if len(disaggregation_axes) == 1:
            disaggregation_data = {disaggregation_axes[0]: subgroup}
        else:
            disaggregation_data = {disaggregation_axes[i]: subgroup[i] for i in range(len(disaggregation_axes))}

        compartments = transitions_data[
            (transitions_data[disaggregation_axes] == pd.Series(disaggregation_data)).all(axis=1)
        ].groupby(
            disaggregation_axes + ['compartment'],
            as_index=False
        ).count()[disaggregation_axes + ['compartment']]

        if 'jail' not in compartments.compartment.unique():
            transitions_data = transitions_data.append(
                {
                    **disaggregation_data,
                    "compartment_duration": 1,
                    "compartment": "jail",
                    "outflow_to": "release",
                    "total_population": 1
                },
                ignore_index=True,
            )

        if 'prison' not in compartments.compartment.unique():
            transitions_data = transitions_data.append(
                {
                    **disaggregation_data,
                    "compartment_duration": 0,
                    "compartment": "prison",
                    "outflow_to": "release",
                    "total_population": 1
                },
                ignore_index=True,
            )



In [ ]:
# # STORE DATA
# upload_spark_model_inputs(
#     "recidiviz-staging",
#     "VA_2022_testing",
#     outflows_data,
#     transitions_data,
#     pd.DataFrame(),
#     "recidiviz/calculator/modeling/population_projection/state/VA/2022/VA_2022_testing_model_inputs.yaml",
# )

In [17]:
outflows_data

,time_step,outflow_to,crime_type,total_population,compartment
0,-36,jail,ASSAULT,22.416667,pretrial
1,-36,jail,BURGLARY/DWELLING,8.250000,pretrial
2,-36,jail,BURGLARY/OTHER,4.000000,pretrial
3,-36,jail,DRUG OTHER,31.833333,pretrial
4,-36,jail,DRUG/SCHEDULE I/II,180.666667,pretrial
...,...,...,...,...,...
95,-1,prison,OTHER SEXUAL ASSAULT,20.500000,pretrial
96,-1,prison,RAPE,14.166667,pretrial
97,-1,prison,ROBBERY,39.083333,pretrial
98,-1,prison,TRAFFIC FELONY,80.916667,pretrial


In [25]:
transitions_data

,compartment,compartment_duration,crime_type,total_population,outflow_to
0,jail,0.032854,ASSAULT,6.0,release
1,jail,0.032854,BURGLARY/DWELLING,1.0,release
2,jail,0.032854,BURGLARY/OTHER,1.0,release
3,jail,0.032854,DRUG OTHER,35.0,release
4,jail,0.032854,DRUG/SCHEDULE I/II,201.0,release
...,...,...,...,...,...
2975,release,360.000000,OTHER SEXUAL ASSAULT,1.0,release
2976,release,360.000000,RAPE,1.0,release
2977,release,360.000000,ROBBERY,1.0,release
2978,release,360.000000,TRAFFIC FELONY,1.0,release


In [26]:
total_population_data

NameError: name 'total_population_data' is not defined